In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import torchvision
import torchvision.transforms as transforms
torch.set_printoptions(linewidth=120)

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [13]:
train_set=torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST',train=True,download=True,transform=transforms.Compose([transforms.ToTensor()])
)

In [5]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [6]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        #convulution layer
        self.conv1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
        self.conv2=nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)
        #linear layer
        self.fc1=nn.Linear(in_features=12*4*4,out_features=120)
        self.fc2=nn.Linear(in_features=120,out_features=60)
        self.out=nn.Linear(in_features=60,out_features=10)


    def forward(self,t):
        #input layer
        t=t
        #conv layer
        t=self.conv1(t)
        t=F.relu(t)
        t=F.max_pool2d(t,kernel_size=2,stride=2)
        t=self.conv2(t)
        t=F.relu(t)
        t=F.max_pool2d(t,kernel_size=2,stride=2)
        #4th hidden layer
        t=t.reshape(-1,12*4*4)
        t=self.fc1(t)
        t=F.relu(t)
        #5th hidden layer
        t=self.fc2(t)
        t=F.relu(t)
        #6th layer
        t=self.out(t)
        return t


In [7]:
sample=next(iter(train_set))
image,label=sample

In [8]:
torch.set_grad_enabled(True)

In [10]:
net=Network()

In [11]:
train_loader=torch.utils.data.DataLoader(train_set,batch_size=100)
optimizer=optim.Adam(net.parameters(),lr=0.01)
total_loss=0
total_correct=0

In [16]:
for epoch in range(5):
    total_loss=0
    total_correct=0
    for batch in train_loader:
        images,labels=batch
        preds=net(images)
        loss=F.cross_entropy(preds,labels)#claculate loss
        optimizer.zero_grad() #removing previous grad
        loss.backward(retain_graph=True) #calculationg grads
        optimizer.step() #updationg weight
        total_loss=total_loss+loss.item()
        total_correct=total_correct+get_num_correct(preds,labels)
    print("epoch:",epoch,"total_correct:",total_correct,"loss:",total_loss)

epoch: 0 total_correct: 52496 loss: 201.48514249920845
epoch: 1 total_correct: 52683 loss: 196.6186412870884
epoch: 2 total_correct: 52657 loss: 195.8144202530384
epoch: 3 total_correct: 52928 loss: 190.96414948999882
epoch: 4 total_correct: 52996 loss: 191.25941368937492


In [17]:
total_correct/len(train_set)

0.8832666666666666